# From N-Grams to Transformer-Based Language Models: Theoretical Foundation

When talking about language models, we are talking about models that assign probabilities to a sequence of tokens: 
$$
p(w_1, w_2, \ldots, w_M ), \text{ with } w_m \in V. \text{ The set } V \text{ is a discrete vocabulary,}
V = \{\text{aardvark, abacus, } \ldots, \text{ zither}\}.
$$

So one might ask why would you want to compute the probability of a word sequence? Well, in many applications the goal is to produce word sequences as output: 

- Machine Translation
- Speech Recognition
- Summarization
- Dialogue Systems (Chat bots), etc

In many of the systems for performing these tasks, there is a subcomponent that compputes the probability of the output text. The purpose of this component is to generate texts that are more fluent. 



## 1. Naive Approach: Unbiasead Language Model

Let's talk about a very __naive approach to build a language model__. A very naive language model would be one, for example that explores the concept of __relative frequency estimate__ to compute the probability of a sequence of tokens. 

Let's work through an example to see this concept in action. Consider the sentence: "Computers are useless, they can only give you answers." Now let's estimate the probability of this sequence of word tokens using the relative frequency estimate: 

$$
p(\text{Computers are useless, they can only give you answers})
= \frac{\text{count}(\text{Computers are useless, they can only give you answers}) \ }{\text{count}(\text{all sentences ever spoken})}
$$

This way of modeling language has a very serious problem, in the theoretical limit of infinite data it would be indeed a good solution, however it is very hard to estimate the count of all the sentences ever spoken, one cannot even imagine how big the dataset would have to be to have an accurate count of all the sentences ever spoken, because of this phenomenon this estimator is said to be __unbiased__. 

Another thing to notice about this implementation is that even grammatically correct sentences would have very low probabilities if they are not included in the set of _all sentences ever spoken_ (i.e., in case we can group them in a dataset). Clearly, this estimator is very data-hungry, and suffers from high vari- ance: even grammatical sentences will have probability zero if they have not occurred in the training data(_side note_:Chomsky famously argued that this is evidence against the very concept of probabilistic language mod- els: no such model could distinguish the grammatical sentence colorless green ideas sleep furiously from the ungrammatical permutation furiously sleep ideas green colorless.)

Therefore what is the solution to this problem? And how can we solve it, __we need to  to introduce bias to have a chance of making reliable estimates from finite training data__. 

## 2. N-grams Language Models

The n-gram language model on the other hand __computes the probability of sequence of tokens as the product of probability of subsequences__. 

$$
\text{The probability of a sequence } p(\mathbf{w}) = p(w_1, w_2, \ldots, w_M) \text{ can be refactored using the chain rule}
$$

$$
p(\mathbf{w}) = p(w_1, w_2, \ldots, w_M) \quad \text{} = p(w_1) \times p(w_2 \mid w_1) \times p(w_3 \mid w_2, w_1) \times \ldots \times p(w_M \mid w_{M-1}, w_1) \quad \text{}
$$


Each element in the product is the probability of a word given all its predecessors. We can think of this as a _word prediction task_: given the context _Computers are_, we want to compute a probability over the next token. The relative frequency estimate of the probability of the word _useless_ in this context is,


$$
p(\text{useless} \mid \text{computers are}) = \frac{\text{count(computers are useless } )}{\sum_{x \in V} \text{count(computers are } x)}
$$

$$
= \frac{\text{count(computers are useless)}}{\text{count(computers are)}}
$$

If you think carefully about the denominator you can see that we haven't really made any progress so far. To computer the conditional probability $$
p(w_M \mid w_{M-1}, w_{M-2}, \ldots, w_1)
$$
 
we would need to model $$
V^{M-1}
$$ contexts


To solve this problem n-grams models make a very interesting assumption, __they condition only on the past n-1 words__: 

$$
p(w_m \mid w_{m-1}, \ldots, w_1) \approx p(w_m \mid w_{m-1}, \ldots, w_{m-n+1})
$$

This means that the probability of a sentence can be approximate as: 

$$
\text{This model requires estimating and storing the probability of only } V^n \text{ events, which is exponential in the order of the n-gram, and not } V^M, \text{ which is exponential in the length of the sentence.}
$$

$$
\text{The n-gram probabilities can be computed by relative frequency estimation,}
$$

$$
p(w_m \mid w_{m-1}, w_{m-2}) = \frac{\text{count}(w_{m-2}, w_{m-1}, w_m)}{\sum_{w'} \text{count}(w_{m-2}, w_{m-1}, w')}
\quad \text{[6.12]}
$$

The hyperparameter  _n_  controls the size of the context used in each conditional probability. If this is misspecified, the language model will perform poorly. Let’s consider the potential problems concretely.


- When n is too small. Consider the following sentences:
  - __Gorillas__ always like to groom their __friends__.
  - The __computer__ that’s on the 3rd floor of our office building __crashed__.

In each example, the words written in bold depend on each other: the likelihood of __their__ depends on knowing that __gorillas__ is plural, and the likelihood of __crashed__ depends on knowing that the subject is a __computer__. _If the n-grams are not big enough to capture this context, then the resulting language model would offer probabilities that are too low for these sentences, and too high for sentences that fail basic linguistic tests like number agreement_.
- When n is too big.
In this case, it is hard good estimates of the n-gram parameters from our dataset, because of data sparsity. To handle the gorilla example, it is necessary to model 6-grams, which means accounting for V 6 events. Under a very small vocab- ulary of V = 104, this means estimating the probability of 1024 distinct events.

These two problems point to another bias-variance tradeoff (see § 2.2.4). A small n- gram size introduces high bias, and a large n-gram size introduces high variance. We can even have both problems at the same time! Language is full of long-range dependen- cies that we cannot capture because n is too small; at the same time, language datasets are full of rare phenomena, whose probabilities we fail to estimate accurately because n is too large. 

#### One solution is to try to keep _n_ large, while still making low-variance estimates of the underlying parameters. To do this, we will introduce a different sort of bias: smoothing.